In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from google.colab import files
import io

# Uploading the file
uploaded = files.upload()

Saving online_retail_II.csv to online_retail_II.csv


In [7]:
try:
    # Getting the filename
    filename = list(uploaded.keys())[0]


    print(f"Reading {filename}...")
    df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='unicode_escape')

except Exception as e:
    print(f"Error loading file: {e}")

    df = pd.read_csv(io.BytesIO(uploaded[filename]))


df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# 4. Preview
print("\n--- COLUMN NAMES ---")
print(list(df.columns))

print("\n--- NULL VALUES ---")
print(df.isnull().sum())

print("\n--- DATA TYPES ---")
print(df.dtypes)

print("\n--- SAMPLE DATA ---")
print(df.head(3))

Reading online_retail_II.csv...

--- COLUMN NAMES ---
['invoice', 'stockcode', 'description', 'quantity', 'invoicedate', 'price', 'customer_id', 'country']

--- NULL VALUES COUNT ---
invoice             0
stockcode           0
description      4382
quantity            0
invoicedate         0
price               0
customer_id    243007
country             0
dtype: int64

--- DATA TYPES ---
invoice         object
stockcode       object
description     object
quantity         int64
invoicedate     object
price          float64
customer_id    float64
country         object
dtype: object

--- SAMPLE DATA (First 3 Rows) ---
  invoice stockcode                          description  quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   

           invoicedate  price  customer_id         country  
0  2009-12-01 07:45:00   6.95   

In [8]:
# --- COLUMN 1: CUSTOMER ID ---
print(f"Original Rows: {len(df)}")

# 1. Dropping Null Customer IDs
df = df.dropna(subset=['customer_id'])

# 2. Convert to Integer
df['customer_id'] = df['customer_id'].astype(int).astype(str)

print(f"Rows after dropping null IDs: {len(df)}")


# --- COLUMN 2: INVOICE DATE ---

# 1. Convert to DateTime Object

df['invoicedate'] = pd.to_datetime(df['invoicedate'])

print("Date conversion successful.")
print(df.dtypes)

Original Rows: 1067371
Rows after dropping null IDs: 824364
Date conversion successful.
invoice                object
stockcode              object
description            object
quantity                int64
invoicedate    datetime64[ns]
price                 float64
customer_id            object
country                object
dtype: object


In [12]:
df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [10]:
# --- COLUMNS 3 & 4: QUANTITY AND PRICE ---


print("--- PRE-CLEAN STATS ---")
print(df[['quantity', 'price']].describe())

# 2. Remove "C" Invoices (C=Cancellations)
# Returns usually start with 'C' in the Invoice number
# We convert to string first to be safe
df['invoice'] = df['invoice'].astype(str)
df = df[~df['invoice'].str.startswith('C')]

# 3. Remove Negative Quantities & Zero Prices
# (Sometimes returns don't have 'C', so we check quantity < 0 too)
df = df[df['quantity'] > 0]
df = df[df['price'] > 0]

# 4. Creating the "Monetary" Feature i.e (Total Price)
df['total_price'] = df['quantity'] * df['price']

print(f"\nRows after removing returns & errors: {len(df)}")
print("\n--- FINAL REVENUE CHECK ---")
print(f"Total Revenue in dataset: {df['total_price'].sum():,.2f}")
print(df[['quantity', 'price', 'total_price']].describe())

--- PRE-CLEAN STATS ---
            quantity          price
count  824364.000000  824364.000000
mean       12.414574       3.676800
std       188.976099      70.241388
min    -80995.000000       0.000000
25%         2.000000       1.250000
50%         5.000000       1.950000
75%        12.000000       3.750000
max     80995.000000   38970.000000

Rows after removing returns & errors: 805549

--- FINAL REVENUE CHECK ---
Total Revenue in dataset: 17,743,429.18
            quantity          price    total_price
count  805549.000000  805549.000000  805549.000000
mean       13.290522       3.206561      22.026505
std       143.634088      29.199173     224.041928
min         1.000000       0.001000       0.001000
25%         2.000000       1.250000       4.950000
50%         5.000000       1.950000      11.850000
75%        12.000000       3.750000      19.500000
max     80995.000000   10953.500000  168469.600000


In [11]:
# --- COLUMN 5: DESCRIPTION CLEANING ---

# 1. Inspect Nulls
print(f"Null Descriptions Before: {df['description'].isnull().sum()}")

# 2. Fill Nulls with a placeholder
df['description'] = df['description'].fillna('UNKNOWN ITEM')

# 3. Standardize to Uppercase & Strip Spaces
# (This merges 'Lunch Bag' and 'LUNCH BAG ' into one)
df['description'] = df['description'].str.upper().str.strip()

# 4. Senior Analyst Tip: Identify "Service" items
# Sometimes 'POSTAGE' or 'MANUAL' are listed as products.
# Let's check the top descriptions to see if they look clean.
print("\n--- TOP 10 MOST SOLD ITEMS ---")
print(df['description'].value_counts().head(10))

print("\n--- SAMPLE CLEAN DATA ---")
print(df[['stockcode', 'description']].head())

Null Descriptions Before: 0

--- TOP 10 MOST SOLD ITEMS ---
description
WHITE HANGING HEART T-LIGHT HOLDER    5181
REGENCY CAKESTAND 3 TIER              3428
ASSORTED COLOUR BIRD ORNAMENT         2777
JUMBO BAG RED RETROSPOT               2702
REX CASH+CARRY JUMBO SHOPPER          2141
PARTY BUNTING                         2121
LUNCH BAG  BLACK SKULL.               2117
LUNCH BAG SPACEBOY DESIGN             1941
HOME BUILDING BLOCK WORD              1929
STRAWBERRY CERAMIC TRINKET BOX        1922
Name: count, dtype: int64

--- SAMPLE CLEAN DATA ---
  stockcode                          description
0     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS
1    79323P                   PINK CHERRY LIGHTS
2    79323W                  WHITE CHERRY LIGHTS
3     22041          RECORD FRAME 7" SINGLE SIZE
4     21232       STRAWBERRY CERAMIC TRINKET BOX


In [13]:
import datetime as dt

# 1. Setting the Snapshot Date i.e (The day after the last invoice)
snapshot_date = df['invoicedate'].max() + dt.timedelta(days=1)

# 2. Aggregation: Group by Customer ID
rfm = df.groupby('customer_id').agg({
    'invoicedate': lambda x: (snapshot_date - x.max()).days, # Recency
    'invoice': 'nunique',                                   # Frequency
    'total_price': 'sum'                                    # Monetary
})

# Renaming columns
rfm.rename(columns={'invoicedate': 'Recency',
                    'invoice': 'Frequency',
                    'total_price': 'Monetary'}, inplace=True)

# 3. Scoring (Quintiles 1-5)
# Recency: Lower is better (5 = bought recently)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency: Higher is better (5 = bought often)
# We use .rank(method='first') to handle tie-breaking for customers with 1 purchase
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetary: Higher is better (5 = spent lots)
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# 4. Create Segment Labels

seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

rfm['Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str)
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

# 5. Reset Index (so customer_id is a column, not index)
rfm.reset_index(inplace=True)


print("--- RFM SEGMENTS PREVIEW ---")
print(rfm[['customer_id', 'Segment', 'Recency', 'Monetary']].head())

rfm.to_csv('RFM_Segments.csv', index=False)

--- RFM SEGMENTS PREVIEW ---
  customer_id              Segment  Recency  Monetary
0       12346           Can't Lose      326  77556.46
1       12347            Champions        2   5633.32
2       12348      Loyal Customers       75   2019.40
3       12349  Potential Loyalists       19   4428.69
4       12350          Hibernating      310    334.40


In [14]:
from google.colab import files

# 1. Export the Clean Transactions (for Drill-Through)
df.to_csv('Clean_Transactions.csv', index=False)

# 2. Download both
print("Downloading RFM_Segments.csv...")
files.download('RFM_Segments.csv')

print("Downloading Clean_Transactions.csv...")
files.download('Clean_Transactions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>